In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

examples = [
    {
        "movie": "Top gun",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "The Godfather",
        "answer": "🧑‍🧑‍🧒🔫🍝"
    },
    {
        "movie": "Zootopia",
        "answer": "🐰🦊🚓"
    }
]

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    memory_key="history"
)


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=ChatPromptTemplate.from_messages([
    ("human", "{movie}?"),
    ("ai", "{answer}")
]),
    examples=examples,
)


prompt = ChatPromptTemplate.from_messages([
    ("system", "You must respond using exactly three emojis and nothing else."),
    ("system", "Conversation so far:\n{history}"),
    example_prompt,
    ("human", "{movie}")
])

def load_memory(_): 
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm # you can use several functions in RunnablePassthrough class

def invoke_chain(question):
    result = chain.invoke({
        "movie": question
    })
    memory.save_context({"input": question},{"output": result.content})
    print(result)

invoke_chain("Avatar")
invoke_chain("Titanic")

content='🪐🌿💥'
content='🚢💔🌊'


In [16]:

memory.load_memory_variables({})


{'history': 'Human: Avatar\nAI: 🪐🌿💥\nHuman: Titanic\nAI: 🚢💔🌊'}